In [ ]:
#Q1

In [ ]:
#Q2

In [1]:
#Q3a

A=1
B=1
x=A
y=B
while (x!=0):
    z=x
    x=x/2
print(z)

5e-324


In [2]:
#Q3part3b
import math
A=1
B=1
x=A
y=B
while (x!=math.inf):
    z=x
    x=x*2
print(z)

KeyboardInterrupt: 

In [ ]:
#Q4

d=1
A=1
B=2
x=A
while (x!=B):
    z=x
    d=d/2
    x=x+d
print(2-z)

In [ ]:
#Q5a

import math
def R(a,b,c):
    x1=((-b+(math.sqrt((b**2)-(4*a*c)))))/(2*a)
    x2=((-b-(math.sqrt((b**2)-(4*a*c)))))/(2*a)
    return x1,x2

print(R(0.001,1000,0.001))

In [ ]:
#Q5b

def Z(a,b,c):
    x1=((-b+(math.sqrt((b**2)-(4*a*c))))*(-b-(math.sqrt((b**2)-(4*a*c)))))/((2*a)*(-b-(math.sqrt((b**2)-(4*a*c)))))
    x2=((-b-(math.sqrt((b**2)-(4*a*c))))*(-b+(math.sqrt((b**2)-(4*a*c)))))/((2*a)*(-b+(math.sqrt((b**2)-(4*a*c)))))
    return x1,x2

print(Z(0.001,1000,0.001))


In [ ]:
#Q6a

def f(x):
    return x*(x-1)

x=1
d= 0.01
derivate = (f(x + d) - f(x))/d
print(derivate)

In [ ]:
#Q6b

A=[10**-4,10**-6,10**-8,10**-10,10**-12,10**-14]
def f(x):
    return x*(x-1)
for d in A:
    x=1
    derivate = (f(x + d) - f(x))/d
    print("the derivate is", derivate)   

In [ ]:
#Q7
